### Initialization

In [23]:
import pandas as pd
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [24]:
df = pd.read_csv('20.csv', dtype=str)
df.head()

,งวดวันที่,งวดเดือน,ปี,เลขหวยหน้า 3 ตัว,เลขหวยหลัง 3 ตัว,เลขท้าย 2 ตัว,Non-breaking space1,เลขหน้า 3 ตัว1,เลขท้าย 3 ตัว2,เลขท้าย 3 ตัว1,เลขหน้า 3 ตัว2,Non-breaking space2,Unnamed: 12
0,1,กันยายน,2559,638,"<font class=""brown"">6<u>84</u></font>",62,&nbsp;,<b>334</b>,<b>335</b>,630,<b>669</b>,&nbsp;,NaN
1,16,สิงหาคม,2559,254,"<font class=""brown""><b>0<u>04</u></b></font>",<b>33</b>,&nbsp;,<b>366</b>,596,631,<b>966</b>,&nbsp;,NaN
2,1,สิงหาคม,2559,272,"<font class=""brown"">9<u>32</u></font>",57,&nbsp;,472,538,871,983,&nbsp;,NaN
3,16,กรกฎาคม,2559,449,"<font class=""brown"">7<u>64</u></font>",<b>55</b>,&nbsp;,158,<b>552</b>,734,925,&nbsp;,NaN
4,1,กรกฎาคม,2559,082,"<font class=""brown"">4<u>60</u></font>",53,&nbsp;,169,173,302,609,&nbsp;,NaN


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   งวดวันที่            480 non-null    object
 1   งวดเดือน             480 non-null    object
 2   ปี                   480 non-null    object
 3   เลขหวยหน้า 3 ตัว     480 non-null    object
 4   เลขหวยหลัง 3 ตัว     480 non-null    object
 5   เลขท้าย 2 ตัว        480 non-null    object
 6   Non-breaking space1  480 non-null    object
 7   เลขหน้า 3 ตัว1       480 non-null    object
 8   เลขท้าย 3 ตัว2       480 non-null    object
 9   เลขท้าย 3 ตัว1       480 non-null    object
 10  เลขหน้า 3 ตัว2       480 non-null    object
 11  Non-breaking space2  480 non-null    object
 12  Unnamed: 12          0 non-null      object
dtypes: object(13)
memory usage: 48.9+ KB


### Data Cleaning

In [26]:
# remove unnecessary columns
df = df.drop(columns=['Non-breaking space1','Non-breaking space2','Unnamed: 12'])

In [27]:
def remove_html_tags(text):
    """ Remove all HTML tags from a text"""
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

In [28]:
# remove tags
for col in df.columns:
    df[col] = df[col].apply(remove_html_tags)

In [29]:
month_dict = {
    'มกราคม': '1',
    'กุมภาพันธ์': '2',
    'มีนาคม': '3',
    'เมษายน': '4',
    'พฤษภาคม': '5',
    'มิถุนายน': '6',
    'กรกฎาคม': '7',
    'สิงหาคม': '8',
    'กันยายน': '9',
    'ตุลาคม': '10',
    'พฤศจิกายน': '11',
    'ธันวาคม': '12'
}

# Replace month names with numbers
df['งวดเดือน'] = df['งวดเดือน'].map(month_dict)

In [30]:
# Create a datetime64 column
df = df.rename(columns={'งวดวันที่' : 'day',
                        'งวดเดือน' : 'month',
                        'ปี' : 'year'})
df[['year', 'month', 'day']] = df[['year', 'month', 'day']].apply(pd.to_numeric)
df['year'] = df['year']-543
df['Date'] = pd.to_datetime(df[['year', 'month', 'day']], format='mixed')

### Feature Engineering

In [31]:
df['รางวัลที่หนึ่ง'] = df['เลขหวยหน้า 3 ตัว'] + df['เลขหวยหลัง 3 ตัว']
df = df.drop(columns = ['เลขหวยหน้า 3 ตัว','เลขหวยหลัง 3 ตัว'])

In [32]:
# split เลขท้าย 2 ตัว into each digit
df['เลขท้าย 2 ตัว หลักสอง'] = pd.to_numeric(df['เลขท้าย 2 ตัว']) // 10
df['เลขท้าย 2 ตัว หลักแรก'] = pd.to_numeric(df['เลขท้าย 2 ตัว']) % 10

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   day                    480 non-null    int64         
 1   month                  480 non-null    int64         
 2   year                   480 non-null    int64         
 3   เลขท้าย 2 ตัว          480 non-null    object        
 4   เลขหน้า 3 ตัว1         480 non-null    object        
 5   เลขท้าย 3 ตัว2         480 non-null    object        
 6   เลขท้าย 3 ตัว1         480 non-null    object        
 7   เลขหน้า 3 ตัว2         480 non-null    object        
 8   Date                   480 non-null    datetime64[ns]
 9   รางวัลที่หนึ่ง         480 non-null    object        
 10  เลขท้าย 2 ตัว หลักสอง  480 non-null    int64         
 11  เลขท้าย 2 ตัว หลักแรก  480 non-null    int64         
dtypes: datetime64[ns](1), int64(5), object(6)
memory usage: 45.1+ KB

### EDA

In [34]:
occurrence = {}

for digit in range(10):
    # Count occurrences of the digit in the 'เลขหวยหน้า 3 ตัว' column
    count = df['รางวัลที่หนึ่ง'].astype(str).str.count(str(digit)).sum()
    occurrence[digit] = count
    
occurrence_series = pd.Series(occurrence)

# Convert the data into a DataFrame with columns for digit and count
occurrence_df = pd.DataFrame({
    'Digit': occurrence_series.index.astype(str),
    'Count': occurrence_series.values
})

# Create the pie chart using Plotly Express
fig = px.pie(occurrence_df, names='Digit', values='Count', title="Occurrence of each number in first prize")

# Show the pie chart
fig.show()

In [35]:
# Initialize a dictionary to store occurrences of each number in each digit position
occurrences = {i: {str(j): 0 for j in range(10)} for i in range(6)}

# Iterate over each lottery number
for number in df['รางวัลที่หนึ่ง']:
    # Iterate over each digit position
    for i, digit in enumerate(number):
        occurrences[i][digit] += 1

# Convert the dictionary into a DataFrame
df_occurrences = pd.DataFrame(occurrences)

# Reshape the DataFrame to have columns for digits and digit positions
df_occurrences = df_occurrences.melt(var_name='Position', value_name='Occurrences', ignore_index=False)

# Reset the index to make 'Position' a column
df_occurrences.reset_index(inplace=True)

# Convert the 'Position' column to numeric for correct ordering
df_occurrences['Position'] = pd.to_numeric(df_occurrences['Position'])

# Sort the DataFrame by 'Position' and 'index' (digit)
df_occurrences.sort_values(by=['Position', 'index'], inplace=True)

# Plot the grouped bar chart using Plotly Express
fig = px.bar(df_occurrences, x='Position', y='Occurrences', color='index', title='Occurrence of each number in each digit position of the first prize (6 digits)',
             labels={'index': 'Number', 'Position': 'Digit Position', 'Occurrences': 'Occurrences'})
fig.update_layout(xaxis=dict(tickvals=list(range(6)), ticktext=[f'Digit {i+1}' for i in range(6)]))

# Show the plot
fig.show()

d:\Study\Year2_2\CPE232\code\.venv\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [36]:
# Create heatmap plot
heatmap_data = df.groupby(['เลขท้าย 2 ตัว หลักสอง', 'เลขท้าย 2 ตัว หลักแรก']).size().unstack(fill_value=0)
heatmap_data
fig = go.Figure(data=go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale='Viridis'))

# Update layout
fig.update_layout(
    title='Heatmap of Winning Numbers',
    xaxis_title='First Digit',
    yaxis_title='Second Digit'
)

# Show plot
fig.show()

In [37]:
# count each number occurence in all prizes
occurrences = {i: {str(j): 0 for j in range(10)} for i in range(6)}
print(occurrences)

for prizes in ['เลขท้าย 2 ตัว']:
    # Iterate over each lottery number
    for number in df[prizes]:
        # Iterate over each digit position
        for i, digit in enumerate(number):
            # print(i,digit)
            occurrences[i+4][digit] += 1
            
for prizes in ['เลขหน้า 3 ตัว1','เลขหน้า 3 ตัว2']:
    # print(prizes)
    # Iterate over each lottery number
    for number in df[prizes]:
        # Iterate over each digit position
        for i, digit in enumerate(number):
            occurrences[i][digit] += 1

for prizes in ['เลขท้าย 3 ตัว1','เลขท้าย 3 ตัว2']:
    # print(prizes)
    # Iterate over each lottery number
    for number in df[prizes]:
        # Iterate over each digit position
        for i, digit in enumerate(number):
            occurrences[i+3][digit] += 1
            
for number in df['รางวัลที่หนึ่ง']:
    # Iterate over each digit position
    for i, digit in enumerate(number):
        occurrences[i][digit] += 1

# Convert the dictionary into a DataFrame
df_occurrences = pd.DataFrame(occurrences)
df_occurrences = df_occurrences.melt(var_name='Position', value_name='Occurrences', ignore_index=False)
df_occurrences.reset_index(inplace=True)
df_occurrences['Position'] = pd.to_numeric(df_occurrences['Position'])
df_occurrences.sort_values(by=['Position', 'index'], inplace=True)
fig = px.bar(df_occurrences, x='Position', y='Occurrences', color='index', title='Occurrence of each number in each digit position of the every prize',
             labels={'index': 'Number', 'Position': 'Digit Position', 'Occurrences': 'Occurrences'},barmode='group')
fig.update_layout(xaxis=dict(tickvals=list(range(6)), ticktext=[f'Digit {i+1}' for i in range(6)]))
fig.show()

{0: {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0}, 1: {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0}, 2: {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0}, 3: {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0}, 4: {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0}, 5: {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0}}


d:\Study\Year2_2\CPE232\code\.venv\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

